<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8N%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%86%8C%EB%B6%84%EB%A5%98_%ED%95%99%EC%8A%B5%EA%B2%B0%EA%B3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 160 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595743 sha256=3acbb0cdf924e800056103499d7c3a9fb0d923a48b26bc71e1130795920fd483
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.4 MB/s 
     |████████████████████████████████| 769 kB 4.2 MB/s 
     |████████████████████████████████| 3.0 MB 62.5 MB/s 
     |████████████████████████████████| 895 kB 58.0 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ce1ba5t1
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본   산업분류자동화	'Colab Notebooks'  'My Drive'


In [4]:
# spell check한 data
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/N_spell_check.csv', sep = ",", encoding = "UTF-8")
ts=pd.read_csv('/mydrive/산업분류자동화/답안 작성용 파일.csv', encoding = "euc-kr")

# 증분한 data
#import pandas as pd
#df=pd.read_csv('/mydrive/산업분류자동화/dataset/inc/df_R_inc.csv', sep = ",", encoding = "UTF-8")
#ts=pd.read_csv('/mydrive/산업분류자동화/답안 작성용 파일.csv', encoding = "euc-kr")

In [5]:
df=df.reset_index(drop=True)

## 데이터 전처리

In [6]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000020,N,76,761,영업장에서 일반 고객을 대상 자동차 임대업
1,id_0000122,N,75,752,여행사에서 일반 고객 대상 해외여행알선
2,id_0000150,N,74,742,고객이 원하는 장소 고객 요구에 따라 건물 내부 청소
3,id_0000185,N,74,742,"건물의 내부 외부에서 고객의 요청으로 건물위생관리, 청소 대행"
4,id_0000248,N,75,752,사무실에서 일반인 대상 국. 내외 여행알선 서비스
5,id_0000444,N,76,762,만화 휴게공간에서 일반인 대상 소설 만화 대여
6,id_0000706,N,75,751,"사업장 구직, 구인, 창업가 정보제공, 시설 이용"
7,id_0000862,N,76,762,사업장에서 고객 대상 정수기 대여
8,id_0000939,N,75,751,직업소개소에서 소개수수로 받고 고용알선
9,id_0000992,N,75,759,매장에서 소비자에게 운송화물포장


In [7]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [8]:
df.shape

(17701, 5)

In [9]:
df['digit_2'].value_counts()

75    10424
76     4389
74     2888
Name: digit_2, dtype: int64

In [10]:
df['digit_3'].value_counts().sort_index()

741     783
742    1866
743     239
751    3712
752    2605
753     524
759    3583
761     938
762    1948
763    1482
764      21
Name: digit_3, dtype: int64

# #### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [11]:
df.loc[(df['digit_3'] == 741), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 742), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 743), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 751), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 752), 'digit_3'] = 4  
df.loc[(df['digit_3'] == 753), 'digit_3'] = 5  
df.loc[(df['digit_3'] == 759), 'digit_3'] = 6  
df.loc[(df['digit_3'] == 761), 'digit_3'] = 7  
df.loc[(df['digit_3'] == 762), 'digit_3'] = 8  
df.loc[(df['digit_3'] == 763), 'digit_3'] = 9  
df.loc[(df['digit_3'] == 764), 'digit_3'] = 10  

In [12]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['영업장에서 일반 고객을 대상 자동차 임대업', '7'], ['여행사에서 일반 고객 대상 해외여행알선', '4'], ['고객이 원하는 장소 고객 요구에 따라 건물 내부 청소', '1'], ['건물의 내부 외부에서 고객의 요청으로 건물위생관리, 청소 대행', '1'], ['사무실에서 일반인 대상 국. 내외 여행알선 서비스', '4']]


## Train data & test data

In [13]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
# Setting parameters
max_len = 64
batch_size = 64 #32,16,8로 감소시키기
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [16]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [17]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [18]:
data_train[0]

(array([   2, 2141, 3494, 6926, 6873,  525, 4856, 7178, 2184, 7136,  517,
        5330, 5461,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(14, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 6)

In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기 (class 수 조정)

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/249 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.4305031299591064 train acc 0.125
epoch 1 batch id 201 loss 0.30613934993743896 train acc 0.6557058457711443
epoch 1 train acc 0.7120031851544107


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.96875


  0%|          | 0/249 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.02907567098736763 train acc 0.984375
epoch 5 batch id 201 loss 0.07337093353271484 train acc 0.9886504975124378
epoch 5 train acc 0.9888930722891566


  0%|          | 0/28 [00:00<?, ?it/s]

epoch 5 test acc 0.9681919642857143


In [23]:
#학습 모델 저장
torch.save(model, 'N_ori_so_model.pt')